In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time ## give a delay execution
import warnings
import numpy as np
from urllib.request import urlretrieve
import base64

from PIL import Image
from io import BytesIO

In [2]:
url = "https://www.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
driver = webdriver.Firefox(executable_path='../geckodriver') # define the webdriver (Firefox, PhantomJS, chrome,...)
driver.implicitly_wait(30)  # sticky timeout to wait implicitly wait for an element to be found
driver.get(url) # load the webpage
tab = driver.find_element_by_id("tournament-fixture")
link = tab.find_elements_by_class_name('result')[:4] 
links = [l.find_element_by_tag_name('a').get_attribute("href") for l in link]
driver.quit()

## Collect the heatmap of the match

In [2]:
url = "https://www.whoscored.com/Matches/1376617/Live/France-Ligue-1-2019-2020-Bordeaux-Nimes"
driver = webdriver.Firefox(executable_path='../geckodriver') # define the webdriver (Firefox, PhantomJS, chrome,...)
driver.implicitly_wait(30)  # sticky timeout to wait implicitly wait for an element to be found
driver.get(url) # load the webpage
time.sleep(5)
try:
    driver.find_elements_by_class_name("qc-cmp-button")[1].click()
except:
    pass
ul = driver.find_element_by_id("live-match-options")
ul.find_elements_by_tag_name('li')[3].click()
heatmap = driver.find_element_by_id("heatmap-pitches")
heatmaps = heatmap.find_elements_by_class_name("canvas-container")
heatmaps = [h.find_element_by_tag_name('canvas') for h in heatmaps]
for i, h in enumerate(heatmaps):
    location = h.location
    size = h.size
    driver.execute_script("window.scrollTo(0, {})".format(location['y'])) 
    png = driver.get_screenshot_as_png() # saves screenshot of entire page
    time.sleep(5)
    im = Image.open(BytesIO(png)) # uses PIL library to open image in memory
    left, upper, right, lower = (location['x'], 0, location['x'] + size['width'], size['height'])
    im = im.crop((left, upper, right, lower)) # defines crop points
    im.save('heatmaps/heat_{}.png'.format(i)) # saves new cropped image
    
driver.quit()

## Collect the commentary of the match

In [28]:
def info_comment(item):    
    field = item.get_attribute("data-field")
    period = item.get_attribute("data-period-id")
    minute = item.get_attribute("data-minute")
    second = item.get_attribute("data-second")
    add_time = item.get_attribute("data-expanded-minute")
    player = item.get_attribute("data-player-id")
    team = item.get_attribute("data-team-id")
    type_action = item.get_attribute("data-type")
    commentary = item.find_element_by_class_name("commentary-text").get_attribute("innerText")
    return (field, period, minute, second, add_time, player, team, type_action, commentary)


In [29]:
url = "https://www.whoscored.com/Matches/1376617/Live/France-Ligue-1-2019-2020-Bordeaux-Nimes"
driver = webdriver.Firefox(executable_path='../geckodriver') # define the webdriver (Firefox, PhantomJS, chrome,...)
driver.implicitly_wait(30)  # sticky timeout to wait implicitly wait for an element to be found
driver.get(url) # load the webpage
time.sleep(5)
try:
    driver.find_elements_by_class_name("qc-cmp-button")[1].click()
except:
    pass

ul = driver.find_element_by_id("live-match-options")
ul.find_elements_by_tag_name('li')[1].click()

coms = []

state = ''
while state != 'last-page':
    items = driver.find_elements_by_class_name("commentary-item")
    com = [info_comment(i) for i in items]
    coms.append(com)
    state = driver.find_element_by_id("match-commentary").get_attribute('data-page-name')
    time.sleep(5)
    (driver
     .find_element_by_class_name("page-navigation")
     .find_elements_by_tag_name("span")[2]
     .click())
    print(state)

In [41]:
col = ["field", "period", "minute", "second", "add_time", "player", "team", "type_action", "commentary"]

import pandas as pd

df = pd.DataFrame({col[i]:data for i, data in enumerate(list(zip(*(c for com in coms for c in com))))})

## Collect the commentary of the match

In [41]:
url = "https://www.whoscored.com/Matches/1376850/LiveStatistics/France-Ligue-1-2019-2020-Monaco-Lille"

In [42]:
driver = webdriver.Firefox(executable_path='../geckodriver') # define the webdriver (Firefox, PhantomJS, chrome,...)
driver.implicitly_wait(30)  # sticky timeout to wait implicitly wait for an element to be found
driver.get(url) # load the webpage
time.sleep(5)

In [43]:
try:
    driver.find_elements_by_class_name("qc-cmp-button")[1].click()
except:
    pass

In [47]:
ul = driver.find_element_by_id("live-player-home-options").find_elements_by_tag_name("li")
length = len(ul)
data_dict = {"Summary": [], "Offensive": [], "Defensive": [], "Passing": []}

In [48]:
col = [c.text for c in driver.find_element_by_id("player-table-statistics-head").find_elements_by_tag_name('th')]

In [49]:
col = list(data_dict)
for i, c in enumerate(col):
    driver.find_element_by_id("live-player-home-options").find_elements_by_tag_name("li")[i].click()
    time.sleep(5)
    for tr in driver.find_element_by_tag_name("tbody").find_elements_by_tag_name('tr'):
        data = [] 
        tds = tr.find_elements_by_tag_name('td')
        data.append(tds[0].text)
        data.append(tds[1].get_attribute('class'))
        data.append(tds[2].find_element_by_tag_name("a").text)
        data.append(tds[2].find_elements_by_class_name("player-meta-data")[-2].text)
        data.append(tds[2].find_elements_by_class_name("player-meta-data")[-1].text)
        data += [td.text for td in tds[3:]]
        data_dict[c].append(data)

IndexError: list index out of range

In [ ]:
data_dict['Defensive']

In [ ]:
len(ul)